# Tensor Network

In [1]:
import Tor10
import numpy as np 
import torch 
import copy

## Bond
Bond = local vector space.

In the following, we create a $d=2$ bond to represent $\{|+\rangle, |-\rangle\}$ or $\{\langle +|, \langle -|\}$.

In [2]:
bd = Tor10.Bond(2)
print(bd)

Dim = 2 |
REGULAR :



## Rank-1 tensor
* Ket: inbond.
* Bra: outbond.

In the basis of $\{|+\rangle, |-\rangle\}$, we create
$$
  |+\rangle \rightarrow 
  \left[
    \begin{array}{c}
    1 \\
    0
    \end{array}
  \right], \;
  |-\rangle \rightarrow 
  \left[
    \begin{array}{c}
    0 \\
    1
    \end{array}
  \right].
$$

In the basis of $\{\langle+|, \langle-|\}$, we create
$$
  \langle+| \rightarrow 
  \left[
    \begin{array}{cc}
    1 & 0
    \end{array}
  \right], \;
  \langle-| \rightarrow 
  \left[
    \begin{array}{cc}
    0 & 1
    \end{array}
  \right].
$$

### $|+\rangle$

In [3]:
# |+>
up_ket = Tor10.UniTensor(bonds=[bd], N_inbond=1, name="|+>")
up_ket.SetElem([[1],[0]])
print(up_ket) # issue: should display as column.
up_ket.Print_diagram()

Tensor name: |+>
is_diag    : False
tensor([1., 0.], dtype=torch.float64)

tensor Name : |+>
tensor Rank : 1
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 2           |      
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :



### $|-\rangle$

In [4]:
dn_ket = Tor10.UniTensor(bonds=[bd], N_inbond=1, name="|->")
dn_ket.SetElem([[0],[1]])
print(dn_ket) # issue: should display as column.
dn_ket.Print_diagram()

Tensor name: |->
is_diag    : False
tensor([0., 1.], dtype=torch.float64)

tensor Name : |->
tensor Rank : 1
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 2           |      
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :



### $\langle +|$

In [5]:
# <+|
up_bra = up_ket.Reshape([2], N_inbond=0)
# up_bra.SetName('<+|') # issue: cannot SetName after a tensor is created.
print(up_bra)
up_bra.Print_diagram()

Tensor name: 
is_diag    : False
tensor([1., 0.], dtype=torch.float64)

tensor Name : 
tensor Rank : 1
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
        |           2 |__ 0  
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :



In [7]:
# <-|
dn_bra = dn_ket.Reshape([2], N_inbond=0)
print(dn_bra)
dn_bra.Print_diagram()

Tensor name: 
is_diag    : False
tensor([0., 1.], dtype=torch.float64)

tensor Name : 
tensor Rank : 1
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
        |           2 |__ 0  
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :



### Orthogonality $\langle i|j\rangle=\delta_{ij}$

In [9]:
for bra in [up_bra, dn_bra]:
    for ket in [up_ket, dn_ket]:
        braket = Tor10.Contract(bra, ket)
#         print(braket.GetBlock()) # issue: bug?
        print(braket)

Tensor name: 
is_diag    : False
tensor(1., dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor(0., dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor(0., dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor(1., dtype=torch.float64)



### $|i\rangle\langle j|$

In [12]:
up_ket.SetLabels([0])
dn_bra.SetLabels([1])
print(up_ket.Print_diagram())
print(dn_bra.Print_diagram())

ketbra = Tor10.Contract(up_ket, dn_bra)
print(ketbra)
ketbra.Print_diagram()

tensor Name : |+>
tensor Rank : 1
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 2           |      
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :

None
tensor Name : 
tensor Rank : 1
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
        |           2 |__ 1  
        |             |     
        ---------------     
lbl:1 Dim = 2 |
REGULAR :

None
Tensor name: 
is_diag    : False
tensor([[0., 1.],
        [0., 0.]], dtype=torch.float64)

tensor Name : 
tensor Rank : 2
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 2         2 |__ 1  
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :

lbl:1 Dim = 2 |
REGULAR :



In [13]:
for ket in [up_ket, dn_ket]:
    for bra in [up_bra, dn_bra]:
        ket.SetLabels([0])
        bra.SetLabels([1])
        ketbra = Tor10.Contract(ket, bra)
        print(ketbra)

Tensor name: 
is_diag    : False
tensor([[1., 0.],
        [0., 0.]], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[0., 1.],
        [0., 0.]], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[0., 0.],
        [1., 0.]], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[0., 0.],
        [0., 1.]], dtype=torch.float64)



### Outter product: otimes $\otimes$

In [14]:
# Tor10.Otimes(up_ket, up_bra) # cannot do this

## Spin operators: $S^+, S^-, S^z$

In [30]:
Sp = Tor10.UniTensor(bonds=[bd,bd], N_inbond=1, name="Sp")
Sp.SetElem([0, 1,
            0, 0])
print(Sp)
print(Sp.Print_diagram())

Tensor name: Sp
is_diag    : False
tensor([[0., 1.],
        [0., 0.]], dtype=torch.float64)

tensor Name : Sp
tensor Rank : 2
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 2         2 |__ 1  
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :

lbl:1 Dim = 2 |
REGULAR :

None


In [39]:
Sm = Tor10.UniTensor(bonds=[bd,bd], N_inbond=1, name="Sm")
Sm.SetElem([0, 0,
            1, 0])
print(Sm)
print(Sm.Print_diagram())

Tensor name: Sm
is_diag    : False
tensor([[0., 0.],
        [1., 0.]], dtype=torch.float64)

tensor Name : Sm
tensor Rank : 2
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 2         2 |__ 1  
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :

lbl:1 Dim = 2 |
REGULAR :

None


In [40]:
Sz = Tor10.UniTensor(bonds=[bd,bd], N_inbond=1, name="Sz")
Sz.SetElem([+1, 0,
            0, -1])
print(Sz)
print(Sz.Print_diagram())

Tensor name: Sz
is_diag    : False
tensor([[ 1.,  0.],
        [ 0., -1.]], dtype=torch.float64)

tensor Name : Sz
tensor Rank : 2
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 2         2 |__ 1  
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :

lbl:1 Dim = 2 |
REGULAR :

None


## Two-sites operator via outterproduct: otimes $\otimes$

In [49]:
SpSm = Tor10.linalg.Otimes(Sp, Sm)
SmSp = Tor10.linalg.Otimes(Sm, Sp)
SzSz = Tor10.linalg.Otimes(Sz, Sz)
print(SpSm)
print(SmSp)
print(SzSz)

Tensor name: 
is_diag    : False
tensor([[0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.]], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[ 1.,  0.,  0.,  0.],
        [ 0., -1.,  0., -0.],
        [ 0.,  0., -1., -0.],
        [ 0., -0., -0.,  1.]], dtype=torch.float64)



In [36]:
print(SpSm+SmSp)

Tensor name: 
is_diag    : False
tensor([[0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.]], dtype=torch.float64)



## Two-sites wavefunction

In [48]:
up1_ket = copy.deepcopy(up_ket)
up2_ket = copy.deepcopy(up_ket)

up1_ket.SetLabels([0])
up2_ket.SetLabels([1])
print(up1_ket.Print_diagram())
print(up2_ket.Print_diagram())

up1up2_ket = Tor10.Contract(up1_ket, up2_ket)
print(up1up2_ket) # issue: should be a single column?
up1up2_ket.Print_diagram()

tensor Name : |+>
tensor Rank : 1
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 2           |      
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :

None
tensor Name : |+>
tensor Rank : 1
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    1 __| 2           |      
        |             |     
        ---------------     
lbl:1 Dim = 2 |
REGULAR :

None
Tensor name: 
is_diag    : False
tensor([[1., 0.],
        [0., 0.]], dtype=torch.float64)

tensor Name : 
tensor Rank : 2
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 2           |      
        |             |     
    1 __| 2           |      
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :

lbl:1 Dim = 2 |
REGULAR :



In [53]:
dn1_ket = copy.deepcopy(dn_ket)
dn2_ket = copy.deepcopy(dn_ket)

dn1_ket.SetLabels([0])
dn2_ket.SetLabels([1])
# print(up1_ket.Print_diagram())
# print(up2_ket.Print_diagram())

dn1dn2_ket = Tor10.Contract(dn1_ket, dn2_ket)
print(dn1dn2_ket) # issue: should be a single column?
dn1dn2_ket.Print_diagram()

Tensor name: 
is_diag    : False
tensor([[0., 0.],
        [0., 1.]], dtype=torch.float64)

tensor Name : 
tensor Rank : 2
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 2           |      
        |             |     
    1 __| 2           |      
        |             |     
        ---------------     
lbl:0 Dim = 2 |
REGULAR :

lbl:1 Dim = 2 |
REGULAR :



## SVD

In [127]:
y = Tor10.UniTensor(bonds=[Tor10.Bond(3),Tor10.Bond(4)],N_inbond=1)
y.SetElem([1,1,0,1,
           0,0,0,1,
           1,1,0,0])
print(y)
print(y.Print_diagram())

Tensor name: 
is_diag    : False
tensor([[1., 1., 0., 1.],
        [0., 0., 0., 1.],
        [1., 1., 0., 0.]], dtype=torch.float64)

tensor Name : 
tensor Rank : 2
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 3         4 |__ 1  
        |             |     
        ---------------     
lbl:0 Dim = 3 |
REGULAR :

lbl:1 Dim = 4 |
REGULAR :

None


In [143]:
u,s,vt = Tor10.Svd_truncate(y)
us = Tor10.Contract(u, s)
usvt = Tor10.Contract(us, vt)
err = usvt - y
print(err.Norm())
print(err)

# print(usvt)
# print(usvt.Print_diagram())

# print(u)
# print(s)
# print(vt)

Tensor name: 
is_diag    : False
tensor(8.4489e-16, dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[ 4.4409e-16,  2.2204e-16,  0.0000e+00,  4.4409e-16],
        [ 2.3606e-16, -6.9529e-17,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  4.4409e-16,  0.0000e+00, -1.1102e-16]],
       dtype=torch.float64)



In [145]:
x = Tor10.UniTensor([Tor10.Bond(4), Tor10.Bond(4)], N_inbond=1, name='x')
x.Rand()
print(x)
print(x.Print_diagram())

Tensor name: x
is_diag    : False
tensor([[0.7151, 0.2564, 0.5226, 0.1144],
        [0.4217, 0.0604, 0.2219, 0.0806],
        [0.2516, 0.4563, 0.2322, 0.2803],
        [0.4564, 0.7802, 0.3124, 0.0370]], dtype=torch.float64)

tensor Name : x
tensor Rank : 2
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 4         4 |__ 1  
        |             |     
        ---------------     
lbl:0 Dim = 4 |
REGULAR :

lbl:1 Dim = 4 |
REGULAR :

None


In [166]:
D = 4
x = Tor10.UniTensor([Tor10.Bond(D), Tor10.Bond(D)], N_inbond=1, name='x')
x.Rand()
# print(x)
print(x.Print_diagram())

for Dcut in range(1, D+1):
    u, s, vt = Tor10.Svd_truncate(x, Dcut)
    us = Tor10.Contract(u, s)
    usvt = Tor10.Contract(us, vt)
    err = usvt - x
    print('Dcut={}'.format(Dcut))
    print(err.Norm())

tensor Name : x
tensor Rank : 2
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    0 __| 10       10 |__ 1  
        |             |     
        ---------------     
lbl:0 Dim = 10 |
REGULAR :

lbl:1 Dim = 10 |
REGULAR :

None
Dcut=1
Tensor name: 
is_diag    : False
tensor(2.5717, dtype=torch.float64)

Dcut=2
Tensor name: 
is_diag    : False
tensor(2.1010, dtype=torch.float64)

Dcut=3
Tensor name: 
is_diag    : False
tensor(1.7525, dtype=torch.float64)

Dcut=4
Tensor name: 
is_diag    : False
tensor(1.4238, dtype=torch.float64)

Dcut=5
Tensor name: 
is_diag    : False
tensor(1.1330, dtype=torch.float64)

Dcut=6
Tensor name: 
is_diag    : False
tensor(0.8043, dtype=torch.float64)

Dcut=7
Tensor name: 
is_diag    : False
tensor(0.6174, dtype=torch.float64)

Dcut=8
Tensor name: 
is_diag    : False
tensor(0.3785, dtype=torch.float64)

Dcut=9
Tensor name: 
is_diag    : False
tensor(0.0434, dtype=torch.float64)

Dcut=10
Tensor name: 
is_diag 

### Bell state

In [167]:
Bell1 = up1up2_ket + dn1dn2_ket
print(Bell1)

Tensor name: 
is_diag    : False
tensor([[1., 0.],
        [0., 1.]], dtype=torch.float64)



In [172]:
u, s, vt = Tor10.Svd_truncate(Bell1)

print(s)
print(u)
print(vt)

Tensor name: 
is_diag    : True
tensor([1., 1.], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[1., 0.],
        [0., 1.]], dtype=torch.float64)

Tensor name: 
is_diag    : False
tensor([[1., 0.],
        [0., 1.]], dtype=torch.float64)



In [67]:
y = Tor10.UniTensor(bonds=[Tor10.Bond(3),Tor10.Bond(4)], N_inbond=1)
y.SetElem([1,1,0,1, 
           0,0,0,1,
           1,1,0,0])
print(y)

Tensor name: 
is_diag    : False
tensor([[1., 1., 0., 1.],
        [0., 0., 0., 1.],
        [1., 1., 0., 0.]], dtype=torch.float64)



In [173]:
x = Tor10.From_torch(torch.arange(0.1,2.5,0.1).reshape(2,3,4).to(torch.float64),labels=[6,7,8],N_inbond=1)
x.Print_diagram()

tensor Name : 
tensor Rank : 3
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
    6 __| 2         3 |__ 7  
        |             |     
        |           4 |__ 8  
        |             |     
        ---------------     
lbl:6 Dim = 2 |
REGULAR :

lbl:7 Dim = 3 |
REGULAR :

lbl:8 Dim = 4 |
REGULAR :



In [65]:
factors, core = Tor10.Hosvd(x,order=[7,6,8],bonds_group=[2,1],by_label=True)

In [66]:
core.Print_diagram()

tensor Name : 
tensor Rank : 2
on device   : cpu
is_diag     : False
        ---------------     
        |             |     
   -1 __| 4           |      
        |             |     
   -2 __| 4           |      
        |             |     
        ---------------     
lbl:-1 Dim = 4 |
REGULAR :

lbl:-2 Dim = 4 |
REGULAR :

